>Instituto Federal de Educação, Ciência e Tecnologia
>>Câmpus Campinas<br>
>>D3TOP – Tópicos em Ciência de Dados<br>
>>Prof.: Samuel Botter Martins<br>

## Preprocessing and data clean step

### Environment Config

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
%%capture
!pip install -U spacy
!pip install -U gensim
!pip install -U neattext
!pip install -U pyspellchecker
!pip install -U stanza 
!pip install unidecode
!python -m spacy download pt_core_news_lg

In [ ]:
# Importando bibliotecas e setando parâmetros

import numpy as np
import pandas as pd
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#### Useful Functions

In [ ]:
def remove_cols(df, mask_cmds, inverse=True):
  '''
  Remove colunas de um dado dataframe df, a partir de uma máscara booleana mask.

  Parameters:
    df (pd.Dataframe): dataframe no qual sofrera a operacao de remocao de colunas
    mask_cmds (list): array de expressões a serem avaliadas. As expressões deve retornar array de booleanos contendo True nas posições das colunas que se deseja remover
    inverse (bool): (opcional) - indica se a máscara será invertida ou não. Default: True

  Returns:
    df (pd.Dataframe): dataframe sem as colunas indicadas na máscara
  '''
  mask_exp = mask_cmds.pop(0)
  mask = eval(mask_exp)
  if inverse:
    return remove_cols(df[df.columns[~mask]], mask_cmds) if len(mask_cmds) else df[df.columns[~mask]]
  return remove_cols(df[df.columns[mask]], mask_cmds) if len(mask_cmds) else df[df.columns[mask]]

In [ ]:
def missing_values_table(df):
  '''
  Funcao que retorna o calculo de valores ausentes, absoluto e percentual, das colunas de um dado dataframe pandas

  Parameters:
    df (pd.Dataframe): dataframe que será  avaliado

  Returns:
    df (pd.Dataframe): dataframe contendo nas linhas o nome das colunas do dataframe original, e nas colunas
    a quantidade total de valores ausentes e o seu percentual correspondente.
  '''
  # Total missing values
  mis_val = df.isnull().sum()
  
  # Percentage of missing values
  mis_val_percent = 100 * df.isnull().sum() / len(df)
  
  # Make a table with the results
  mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
  
  # Rename the columns
  mis_val_table_ren_columns = mis_val_table.rename(
  columns = {0 : 'Missing Values', 1 : '% of Total Values'})
  
  # Sort the table by percentage of missing descending
  mis_val_table_ren_columns = mis_val_table_ren_columns[
      mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
  '% of Total Values', ascending=False).round(1)
  
  # Print some summary information
  print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
      "There are " + str(mis_val_table_ren_columns.shape[0]) +
        " columns that have missing values.")
  
  # Return the dataframe with missing information
  return mis_val_table_ren_columns

In [ ]:
def neat_preprocessing(text_in: str, ) -> str:
  import neattext.functions as ntx

  text = text_in.lower()
  
  # text = ntx.fix_contractions(text)
  text = ntx.remove_html_tags(text)
  text = ntx.remove_punctuations(text) 
  text = ntx.replace_term(text, ntx.USER_HANDLES_REGEX, 'USUARIO')  
  text = ntx.remove_urls(text)
  text = ntx.remove_emails(text)
  text = ntx.remove_phone_numbers(text)
  text = ntx.remove_numbers(text)  
  text = ntx.replace_emojis(text, "EMOJI")
  text = ntx.remove_multiple_spaces(text)
  text = ntx.remove_currency_symbols(text)
  # text = ntx.remove_special_characters(text)
  
  return text


### Data gathering

The data gathering procedure was done through [Apify](https://apify.com/dtrungtin/airbnb-scraper/input-schema#locationQuery), a web scraping and automation platform.

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/IFSP/Trabalhos_Grupo_IFSP/NLP

/content/drive/MyDrive/Colab Notebooks/IFSP/Trabalhos_Grupo_IFSP/NLP


In [ ]:
file_path = 'data/raw/'
file_name = 'dataset_airbnb-scraper_2023-04-04_01-45-49-997.csv'

df_raw = pd.read_csv(file_path + file_name, dtype=str)
print(f'No Rows: {df_raw.shape[0]}')
print(f'No Columns: {df_raw.shape[1]}', end='\n\n')
display(df_raw.head())

No Rows: 547
No Columns: 1107



,additionalHosts/0/about,additionalHosts/0/badges/0,additionalHosts/0/firstName,additionalHosts/0/hasInclusionBadge,additionalHosts/0/hasProfilePic,additionalHosts/0/hostUrl,additionalHosts/0/id,additionalHosts/0/isSuperHost,additionalHosts/0/languages/0,additionalHosts/0/languages/1,additionalHosts/0/languages/2,additionalHosts/0/languages/3,additionalHosts/0/languages/4,additionalHosts/0/memberSince,additionalHosts/0/pictureUrl,additionalHosts/0/responseRate,additionalHosts/0/responseTime,additionalHosts/0/smartName,additionalHosts/0/thumbnailUrl,additionalHosts/1/about,additionalHosts/1/badges/0,additionalHosts/1/firstName,additionalHosts/1/hasInclusionBadge,additionalHosts/1/hasProfilePic,additionalHosts/1/hostUrl,additionalHosts/1/id,additionalHosts/1/isSuperHost,additionalHosts/1/languages/0,additionalHosts/1/languages/1,additionalHosts/1/memberSince,additionalHosts/1/pictureUrl,additionalHosts/1/responseRate,additionalHosts/1/responseTime,additionalHosts/1/smartName,additionalHosts/1/thumbnailUrl,additionalHosts/2/about,additionalHosts/2/firstName,additionalHosts/2/hasInclusionBadge,additionalHosts/2/hasProfilePic,additionalHosts/2/hostUrl,additionalHosts/2/id,additionalHosts/2/isSuperHost,additionalHosts/2/memberSince,additionalHosts/2/pictureUrl,additionalHosts/2/smartName,additionalHosts/2/thumbnailUrl,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,name,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,photos/1/caption,photos/1/pictureUrl,photos/1/thumbnailUrl,photos/2/caption,photos/2/pictureUrl,photos/2/thumbnailUrl,photos/3/caption,photos/3/pictureUrl,photos/3/thumbnailUrl,photos/4/caption,photos/4/pictureUrl,photos/4/thumbnailUrl,photos/5/caption,photos/5/pictureUrl,photos/5/thumbnailUrl,photos/6/caption,photos/6/pictureUrl,photos/6/thumbnailUrl,photos/7/caption,photos/7/pictureUrl,photos/7/thumbnailUrl,photos/8/caption,photos/8/pictureUrl,photos/8/thumbnailUrl,photos/9/caption,photos/9/pictureUrl,photos/9/thumbnailUrl,photos/10/caption,photos/10/pictureUrl,photos/10/thumbnailUrl,photos/11/caption,photos/11/pictureUrl,photos/11/thumbnailUrl,photos/12/caption,photos/12/pictureUrl,photos/12/thumbnailUrl,photos/13/caption,photos/13/pictureUrl,photos/13/thumbnailUrl,photos/14/caption,photos/14/pictureUrl,photos/14/thumbnailUrl,photos/15/caption,photos/15/pictureUrl,photos/15/thumbnailUrl,photos/16/caption,photos/16/pictureUrl,photos/16/thumbnailUrl,photos/17/caption,photos/17/pictureUrl,photos/17/thumbnailUrl,photos/18/caption,photos/18/pictureUrl,photos/18/thumbnailUrl,photos/19/caption,photos/19/pictureUrl,photos/19/thumbnailUrl,photos/20/caption,photos/20/pictureUrl,photos/20/thumbnailUrl,photos/21/caption,photos/21/pictureUrl,photos/21/thumbnailUrl,photos/22/caption,photos/22/pictureUrl,photos/22/thumbnailUrl,photos/23/caption,photos/23/pictureUrl,photos/23/thumbnailUrl,photos/24/caption,photos/24/pictureUrl,photos/24/thumbnailUrl,photos/25/caption,photos/25/pictureUrl,photos/25/thumbnailUrl,photos/26/caption,photos/26/pictureUrl,photos/26/thumbnailUrl,photos/27/caption,photos/27/pictureUrl,photos/27/thumbnailUrl,photos/28/caption,photos/28/pictureUrl,photos/28/thumbnailUrl,photos/29/caption,photos/29/pictureUrl,photos/29/thumbnailUrl,photos/30/caption,photos/30/pictureUrl,photos/30/thumbnailUrl,photos/31/caption,photos/31/pictureUrl,photos/31/thumbnailUrl,photos/32/caption,photos/32/pictureUrl,photos/32/thumbnailUrl,photos/33/caption,photos/33/pictureUrl,photos/33/thumbnailUrl,photos/34/caption,photos/34/pictureUrl,photos/34/thumbnailUrl,photos/35/caption,photos/35/pictureUrl,photos/35/thumbnailUrl,photos/36/caption,photos/36/pictureUrl,photos/36/thumbnailUrl,photos/37/caption,photos/37/pictureUrl,photos/37/thumbnailUrl,photos/38/caption,photos/38/pictureUrl,photos/38/thumbnailUrl,photos/39/caption,photos/39/pictureUrl,photos/39/thumbnailUrl,photos/40/caption,photos/40/pictureUrl,photos/40/thumbnailUrl,photos/41/caption,photos/41/pictureUrl,photos/41/thumbnailUrl,photos

### Step 01 - discarding information that doesn't matter

Removing columns relating to:
- `additionalHosts`
- property `photos` (we'll keep just first) 
- URL's
- `primaryHost/languages` (we'll keep just first) 

In [ ]:
mask_additionalHosts = "df.columns.str.startswith('additionalHosts')"
mask_photos = "df.columns.str.contains(r'^photos/[^0]')"
mask_url = "df.columns.str.contains(r'^(?!photos).*Url$')"
mask_phlang = "df.columns.str.contains(r'^primaryHost/languages/[^0]')"
masks_commands = [mask_additionalHosts, mask_photos, mask_url, mask_phlang]

df_s01 = df_raw.pipe(remove_cols, masks_commands)

print(f'No Columns before step 01: {df_raw.shape[1]}')
print(f'No Columns after step 01: {df_s01.shape[1]}', end='\n\n')
df_s01.sample(1)

No Columns before step 01: 1107
No Columns after step 01: 452



,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,name,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/about,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,reviews/0/author/firstName,reviews/0/author/hasProfilePic,reviews/0/author/id,reviews/0/author/smartName,reviews/0/collectionTag,reviews/0/comments,reviews/0/createdAt,reviews/0/id,reviews/0/language,reviews/0/localizedDate,reviews/0/localizedReview,reviews/0/localizedReview/comments,reviews/0/localizedReview/disclaimer,reviews/0/localizedReview/needsTranslation,reviews/0/localizedReview/response,reviews/0/rating,reviews/0/recipient/firstName,reviews/0/recipient/hasProfilePic,reviews/0/recipient/id,reviews/0/recipient/smartName,reviews/0/response,reviews/1/author/firstName,reviews/1/author/hasProfilePic,reviews/1/author/id,reviews/1/author/smartName,reviews/1/collectionTag,reviews/1/comments,reviews/1/createdAt,reviews/1/id,reviews/1/language,reviews/1/localizedDate,reviews/1/localizedReview,reviews/1/localizedReview/comments,reviews/1/localizedReview/disclaimer,reviews/1/localizedReview/needsTranslation,reviews/1/localizedReview/response,reviews/1/rating,reviews/1/recipient/firstName,reviews/1/recipient/hasProfilePic,reviews/1/recipient/id,reviews/1/recipient/smartName,reviews/1/response,reviews/2/author/firstName,reviews/2/author/hasProfilePic,reviews/2/author/id,reviews/2/author/smartName,reviews/2/collectionTag,reviews/2/comments,reviews/2/createdAt,reviews/2/id,reviews/2/language,reviews/2/localizedDate,reviews/2/localizedReview,reviews/2/localizedReview/comments,reviews/2/localizedReview/disclaimer,reviews/2/localizedReview/needsTranslation,reviews/2/localizedReview/response,reviews/2/rating,reviews/2/recipient/firstName,reviews/2/recipient/hasProfilePic,reviews/2/recipient/id,reviews/2/recipient/smartName,reviews/2/response,reviews/3/author/firstName,reviews/3/author/hasProfilePic,reviews/3/author/id,reviews/3/author/smartName,reviews/3/collectionTag,reviews/3/comments,reviews/3/createdAt,reviews/3/id,reviews/3/language,reviews/3/localizedDate,reviews/3/localizedReview,reviews/3/localizedReview/comments,reviews/3/localizedReview/disclaimer,reviews/3/localizedReview/needsTranslation,reviews/3/localizedReview/response,reviews/3/rating,reviews/3/recipient/firstName,reviews/3/recipient/hasProfilePic,reviews/3/recipient/id,reviews/3/recipient/smartName,reviews/3/response,reviews/4/author/firstName,reviews/4/author/hasProfilePic,reviews/4/author/id,reviews/4/author/smartName,reviews/4/collectionTag,reviews/4/comments,reviews/4/createdAt,reviews/4/id,reviews/4/language,reviews/4/localizedDate,reviews/4/localizedReview,reviews/4/localizedReview/comments,reviews/4/localizedReview/disclaimer,reviews/4/localizedReview/needsTranslation,reviews/4/localizedReview/response,reviews/4/rating,reviews/4/recipient/firstName,reviews/4/recipient/hasProfilePic,reviews/4/recipient/id,reviews/4/recipient/smartName,reviews/4/response,reviews/5/author/firstName,reviews/5/author/hasProfilePic,reviews/5/author/id,reviews/5/author/smartName,reviews/5/collectionTag,reviews/5/comments,reviews/5/createdAt,reviews/5/id,reviews/5/language,reviews/5/localizedDate,reviews/5/localizedReview,reviews/5/localizedReview/comments,reviews/5/localizedReview/disclaimer,reviews/5/localizedReview/needsTranslation,reviews/5/localizedReview/response,reviews/5/rating,reviews/5/recipient/firstName,reviews/5/recipient/hasProfilePic,reviews/5/recipient/id,reviews/5/recipient/smartName,reviews/5/response,reviews/6/author/firstName,reviews/6/author/hasProfilePic,reviews/6/author/id,reviews/6/author/smart

### Step 02 - Evaluate Missing Values

In [ ]:
missing_values_table(df_s01)

Your selected dataframe has 452 columns.
There are 429 columns that have missing values.


,Missing Values,% of Total Values
reviews/2/collectionTag,547,100.000
reviews/15/collectionTag,547,100.000
reviews/9/localizedReview,547,100.000
reviews/1/localizedReview,547,100.000
reviews/9/collectionTag,547,100.000
reviews/14/collectionTag,547,100.000
reviews/8/localizedReview,547,100.000
reviews/18/localizedReview/response,547,100.000
reviews/14/localizedReview,547,100.000
reviews/18/localizedReview,547,100.000


### Step 03 - discarding information based on missing values

keeping only the following columns about reviews:
- Review: `comments`, `createdAt`, `id`, `rating`
- Review Author: `firstName`, `id`

In [ ]:
mask_reviews_data = "df.columns.str.contains(r'^reviews/[0-9]+/(?!author/)(?!comments|createdAt|id|rating)')"
mask_author_data = "df.columns.str.contains(r'^reviews/[0-9]+/author/(?!firstName|id)')"
masks_commands = [mask_reviews_data, mask_author_data]

df_s03 = df_s01.pipe(remove_cols, masks_commands)

print(f'No Columns before step 02: {df_s01.shape[1]}')
print(f'No Columns after step 02: {df_s03.shape[1]}', end='\n\n')
df_s03.sample(1)

No Columns before step 02: 452
No Columns after step 02: 152



,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,name,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/about,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,reviews/0/author/firstName,reviews/0/author/id,reviews/0/comments,reviews/0/createdAt,reviews/0/id,reviews/0/rating,reviews/1/author/firstName,reviews/1/author/id,reviews/1/comments,reviews/1/createdAt,reviews/1/id,reviews/1/rating,reviews/2/author/firstName,reviews/2/author/id,reviews/2/comments,reviews/2/createdAt,reviews/2/id,reviews/2/rating,reviews/3/author/firstName,reviews/3/author/id,reviews/3/comments,reviews/3/createdAt,reviews/3/id,reviews/3/rating,reviews/4/author/firstName,reviews/4/author/id,reviews/4/comments,reviews/4/createdAt,reviews/4/id,reviews/4/rating,reviews/5/author/firstName,reviews/5/author/id,reviews/5/comments,reviews/5/createdAt,reviews/5/id,reviews/5/rating,reviews/6/author/firstName,reviews/6/author/id,reviews/6/comments,reviews/6/createdAt,reviews/6/id,reviews/6/rating,reviews/7/author/firstName,reviews/7/author/id,reviews/7/comments,reviews/7/createdAt,reviews/7/id,reviews/7/rating,reviews/8/author/firstName,reviews/8/author/id,reviews/8/comments,reviews/8/createdAt,reviews/8/id,reviews/8/rating,reviews/9/author/firstName,reviews/9/author/id,reviews/9/comments,reviews/9/createdAt,reviews/9/id,reviews/9/rating,reviews/10/author/firstName,reviews/10/author/id,reviews/10/comments,reviews/10/createdAt,reviews/10/id,reviews/10/rating,reviews/11/author/firstName,reviews/11/author/id,reviews/11/comments,reviews/11/createdAt,reviews/11/id,reviews/11/rating,reviews/12/author/firstName,reviews/12/author/id,reviews/12/comments,reviews/12/createdAt,reviews/12/id,reviews/12/rating,reviews/13/author/firstName,reviews/13/author/id,reviews/13/comments,reviews/13/createdAt,reviews/13/id,reviews/13/rating,reviews/14/author/firstName,reviews/14/author/id,reviews/14/comments,reviews/14/createdAt,reviews/14/id,reviews/14/rating,reviews/15/author/firstName,reviews/15/author/id,reviews/15/comments,reviews/15/createdAt,reviews/15/id,reviews/15/rating,reviews/16/author/firstName,reviews/16/author/id,reviews/16/comments,reviews/16/createdAt,reviews/16/id,reviews/16/rating,reviews/17/author/firstName,reviews/17/author/id,reviews/17/comments,reviews/17/createdAt,reviews/17/id,reviews/17/rating,reviews/18/author/firstName,reviews/18/author/id,reviews/18/comments,reviews/18/createdAt,reviews/18/id,reviews/18/rating,reviews/19/author/firstName,reviews/19/author/id,reviews/19/comments,reviews/19/createdAt,reviews/19/id,reviews/19/rating,roomType,stars,url
173,"São Paulo, Brasil",true,false,-23.56498,-46.66281,VERY CHARMING AND PLEASANT- JARDINS,6,NaN,https://a0.muscache.com/im/pictures/miso/Hosting-11989551/original/26166b9c-5ffa-420b-a8fc-34b45dd6fa2a.jpeg?aki_policy=large,https://a0.muscache.com/im/pictures/miso/Hosting-11989551/original/26166b9c-5ffa-420b-a8fc-34b45dd6fa2a.jpeg?aki_policy=small,1800,R$1.800,BRL,false,nightly,"SAO PAULO, SP, BRAZIL\r\nDESIGNER",15 avaliações,Identidade verificada,NaN,Joanna,false,true,2784274,false,English,Membro desde junho de 2012,100%,dentro de um dia,Joanna,JoseLuiz,191072337,Localização e ambientação ótimos!,2019-01-31T18:03:59Z,406936944,5,Suely,21174527,"O apartamento é ultra bem localizado, e estiloso, mas pra esse padrão, falta modernizar a cozinha e inclusive trocar utensílios básicos como talheres de serviço, panelas, tábua, jogo americano etc...<br/>Não tem microondas, nem sanduicheira. E a torradeira, não funcionava.",2018-10-17T18:08:09Z,337802712,4,France,76612342,Joanna é una pessoa muito legal que nos a

### Step 04 - Getting text columns from dataframe

In [ ]:
# Definindo as  colunas de texto
mask_text_cols = "df.columns.str.contains(r'name|about|comments')"
df_txt_cols = df_s03.pipe(remove_cols, [mask_text_cols], inverse=False)
df_txt_cols.sample(1)

,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments,reviews/10/comments,reviews/11/comments,reviews/12/comments,reviews/13/comments,reviews/14/comments,reviews/15/comments,reviews/16/comments,reviews/17/comments,reviews/18/comments,reviews/19/comments
110,Casa maravilhosa 10 per- Jardim Europa/rua Polonia,"Alexandra Loras e Direitora dos filmes “inconscientes revelados” e “cordial”.\n\nConsultora em gestão de performance e mudança organizacional, desenvolveu programa de capacitação para mulheres e afrodescendentes subir na alta liderança das empresas \n\nMestra em Gestão de Mídia pela Sciences Po (Paris). Loras é Fundadora do Fórum Protagonismo Feminino, e palestrante sobre raça, gênero e diversidade. Seu trabalho é focado no desenvolvimento de liderança consciente para catalisar a transformação organizacional sobre diversidade etnico-racial e empoderamento feminino. Conduziu palestras para mais de 10 mil pessoas em todo o mundo, desde TEDxSãoPaulo, até grandes organizações internacionais como Google, JP Morgan e Facebook . \n","Casa muito bonita, bem limpa, excelente localização, e bem segura.",A estadia foi maravilhosa. A comunicação com a Alexandra e com a Eli foi sempre muita fluída e rápida. A casa atende muito bem e da toda a segurança que se busca em São Paulo.,"Tivemos um fim de semana muito feliz na casa de Alexandra. Fomos muito bem recebidos. A casa é, ao mesmo tempo, extremamente aconchegante e ampla. Nossa família é grande e nos acomodamos super bem. A cozinha é bem completa e os quartos proporcionaram sonos deliciosos. Esperamos ter a oportunidade de voltar!","Casa em excelente localização. Espaçosa, completa e confortável. A anfitriã sempre muito prestativa e atenciosa. Local extremamente seguro. Foi uma ótima experiência.","Alexandra foi excelente anfitriã, comunicativa e compreensiva para acordar check in e check out. A casa é fantástica, ideal para jantares/almoços e encontros familiares. Recomendo fortemente!",Tudo perfeito. A casa é ótima e muito bem localizada.,Tudo perfeito,"A casa é muito bonita, exatamente como nas fotos, excelente localização! Alexandra disponibilizou os brinquedos do filho para a crianças e foi um sucesso! Nossa estadia foi muito agradável. Super recomendo essa casa!","A casa é ótima, aconchegante, e bem localizada. Alexandra e Eli são muito simpáticas. Recomendo!!","Casa muito bonita e localização excelente. Alexandra é super atenciosa, voltaremos !","Casa muito bem localizada, bairro seguro, guarita com vigilante 24h bem ao lado.<br/>Espaçosa e confortável. Churrasqueira no patio da frente. As crianças adoraram.","Ótima experiência.<br/><br/>Alexandra e' uma ótima anfitriã, simpática, estava sempre disponível e demonstrou sincero desejo de que nossa estadia fosse perfeita. <br/><br/>Casa bem espaçosa, charmosa e completa, conforme fotos.<br/>Estávamos em 2 adultos e 3 crianças > não sentimos falta de nada. <br/><br/>- Casa: Bem localizada, em bairro tranquilo, arejada e com boa iluminação. ter o guarita próximo trouxe segurança. <br/>- Itens da casa: Casa completa, copos taças e pratos de bom gosto, cobertores bem quentinhos. Internet pegou bem para reuniões nos dois andares da casa e para streamings nos nossos devices e TV.<br/>- Limpeza e conservação: Quando chegamos notamos tudo limpo. Alguns itens novos, outros mais antigos, uns práticos e outros mais requintados, no geral bem conservados.<br/>- Check-in e comunicação: Acabamos chegando mais tarde e a pessoa que nos recebeu foi super atenciosa e conseguiu nos explicar tudo além de deixar o contado dela e do guarita para qualquer eventualidade. Durante toda estadia me senti bem amparada e a proprietária estava sempre disponível. <br/><br/>O único ponto que vale mencionar é o secador de cabelos que estava bem descrito como não disponível no anuncio, mas fica aqui como sug

In [ ]:
arr_texts = df_txt_cols.fillna('').values
arr_texts[:5]

array([['Ap 2q conforto e comodidade!', '',
        'The host canceled this reservation 107 days before arrival. This is an automated posting.',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', ''],
       ['Apartamento Guarujá Praia Enseada Sp (GRUPO REIS)', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', ''],
       ['Casa na Praia de Pernambuco, Guarujá - SP', '',
        'A casa fica bem próxima da praia, além disso a casa é uma graça, tudo muito limpo e organizado. Os anfritriões são muito atenciosos. Recomendo, voltaremos mais vezes!',
        'Um lugar lindo e próximo a praia, da para ir de a pé.  Cristian e dona Maria super atenciosos e cuidadosos conosco. Casa super limpa e com utensílios que dá para cozinhar sem problemas. A praia é de água limpa e bem familiar. Amamos o lugar e pretendemos voltar mais vezes.  Valeu muito a pena !',
        'Cristian e Dona Maria excelentes anfitriões! <br/

### Step 05 - Defining preprocessing and cleaning operations

In [ ]:
from typing import Union
import spacy
spacy.load("pt_core_news_lg")
import stanza
stanza.download('pt')

def preprocess_txt(textraw:str, nlp_obj:Union[spacy.lang.pt.Portuguese, stanza.pipeline.core.Pipeline], lemma=False) -> str:
  '''
  Realiza a limpeza e o pre processamento de um dado texto em portugues.
  As etapas de pré-processamento a serem executadas são:
  * Conversão para Minúsculas.
  * Remocao de tags HTML.
  * Remocao de pontuacoes.
  * Remocao de algarismos numericos e numeros de telefone.
  * Remocao de multiplos espacos.
  * Remocao de URLs e E-mails: são removidos, pois eles não adicionam significado ao texto.
  * Substituicao de Emojis: Substitui emojis usando um dicionário predefinido contendo emojis junto com seu significado. (por exemplo: ":)" para "sorriso")
  * Substituindo nomes de usuário: Substitua @Usernames pela palavra "USUARIO". (por exemplo: "@nick_name" para "USUARIO")
  * Remocao de ascentuacao.
  * Removendo Não-Alfabetos: Substitui caracteres (exceto Dígitos e Alfabetos) por um espaço.
  * Removendo letras consecutivas: 3 ou mais letras consecutivas são substituídas por 2 letras. (por exemplo: "Muitoooo" para "Muitoo")
  * Remoção de palavras curtas: palavras com comprimento menor que 2 são removidas.
  * Removendo Stopwords: Stopwords são as palavras que não adicionam muito significado a uma frase. Eles podem ser ignorados com segurança sem sacrificar o significado da frase. (por exemplo: "the", "he", "have")
  * Lematização: A lematização é o processo de converter uma palavra em sua forma básica. (por exemplo: “Great” para “Good”). A lematização geralmente retorna palavras válidas (que existem) enquanto as técnicas de stemming retornam (na maioria das vezes) palavras abreviadas

  Parameters:
    textraw (str): texto bruto o qual sofrera as operacoes de limpeza e pre processamento.
    nlp_obj (spacy.lang.pt.Portuguese | stanza.pipeline.core.Pipeline): - objeto que indica a biblioteca a ser utilizada para a etapa de tokenizacao, remocao de stop words e lemmatizacao.
    lemma (boolean): (opcional) se deve substituir as palavras originais do texto por seu lemma ou nao. Default: False

  Returns:
    wordsclean (pd.Dataframe): dataframe sem as colunas indicadas na máscara
  '''
  import re
  from unidecode import unidecode
  # from gensim.utils import simple_preprocess
  
    

  # Definindo padrões Regex
  # urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
  # userPattern       = '@[^\s]+'
  alphaPattern      = "[^a-zA-Z0-9]"
  sequencePattern   = r"(.)\1\1+" # letras consecutivas
  seqReplacePattern = r"\1\1"
  
  # Definindo dicionario de emojis e seus respectivos significados
  emojis = {':)': 'sorri', ':-)': 'sorri', ';d': 'pisca', ':-E': 'vampiro', ':(': 'triste', 
            ':-(': 'triste', ':-<': 'triste', ':P': 'mostra lingua', ':O': 'surpresa',
            ':-@': 'chocado', ':@': 'chocado',':-$': 'confuso', ':\\': 'irritado', 
            ':#': 'mudo', ':X': 'mudo', ':^)': 'sorri', ':-&': 'confuso', '$_$': 'ganancia',
            '@@': 'vira os olhos', ':-!': 'confuso', ':-D': 'sorri', ':-0': 'grita', 'O.o': 'confuso',
            '<(-_-)>': 'robo', 'd[-_-]b': 'dj', ":'-)": 'sorri triste', ';)': 'piscar', 
            ';-)': 'piscar', 'O:-)': 'anjo','O*-)': 'anjo','(:-D': 'fofoca', '=^.^=': 'gato'}

   
  # ##################
  # Tentativa de aplicação de técnica para correção de palavras com erro de escrita, 
  # mas o resultado não foi considerado satisfatório.
  # ##################
  # from gensim.utils import simple_tokenize
  # from spellchecker import SpellChecker
  # spell = SpellChecker(language='pt')
  # wordlist = list(simple_tokenize(textraw))
  # misspelled = spell.unknown(wordlist)
  # correctedtext = [spell.correction(word) for word in misspelled]
  # print(correctedtext)

  textclean = neat_preprocessing(textraw)  
  for emoji in emojis.keys():
      textclean = textclean.replace(emoji, "EMOJI" + emojis[emoji])  
  textclean = re.sub(sequencePattern, seqReplacePattern, textclean)
  textclean = unidecode(textclean)
  textclean = re.sub(alphaPattern, " ", textclean)
  wordsclean = ''

  if isinstance(nlp, spacy.lang.pt.Portuguese):
    doc = nlp(textclean)
    for token in doc:      
      if not token.is_stop and len(token) > 1:
        if lemma:
          wordsclean += token.lemma_+' '
        else:
          wordsclean += token.text + ' '

  elif isinstance(nlp, stanza.pipeline.core.Pipeline):
    # 
    doc = nlp(textclean)
    for sent in doc.sentences:
      for word in sent.words:      
        if lemma:
          wordsclean += word.lemma +' '
        else:
          wordsclean += word.text + ' '
  else:
      raise 'Instancia nao reconhecida do objeto `nlp_obj`'
    
  return wordsclean


INFO:stanza:Downloading default packages for language: pt (Portuguese) ...
INFO:stanza:File exists: /root/stanza_resources/pt/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


### Step 06 - Tests

In [ ]:
arr_texts[204]

array(['COBERTURA PANORÂMICA GUARUJÁ PITANGUEIRAS COM SPA', '',
       'Nossa estadia foi maravilhosa , belíssima vista da praia , sem falar na Cida ótima cozinheira , nota 1000 para Risoto !',
       'Adoramos tudo e a Cida uma querida 🤩🙏🏻',
       'O apartamento está bem localizado  , possui uma vista para Mar privilegiada. O atendimento do Flavio e da Cida foi impecável. Super indico',
       'Flávio e Cida foram muito gentis. O apartamento é realmente bom e fica em frente a praia. É lindo! A Jacuzzi não estava aquecendo, porém o proprietário disse estar regularizando.',
       'Apartamento muito bom, aconchegante, ótima localização. A funcionária Cida é maravilhosa, atenciosa e cozinha extremamente bem, ficamos muito satisfeitos.',
       'Flávio muito atencioso e Cida quem nos ajuda no período da estadia é formidável.',
       'Eu e minha família amamos o apartamento do sr. Flávio. Localização maravilhosa, vista linda e com a jacuzzi, churrasqueira e mesa de bilhar não precisamos 

#### Teste Spacy

In [ ]:
%%time
nlp = spacy.load("pt_core_news_lg")
# nlp = stanza.Pipeline('pt')
postprocess = []
for rawtxt in arr_texts[204]:
  if len(rawtxt):
    postprocess.append(preprocess_txt(rawtxt, nlp_obj=nlp, lemma=True))

postprocess

CPU times: user 1.09 s, sys: 338 ms, total: 1.43 s
Wall time: 1.42 s


['cobertura panoramico guaruja pitangueira spa ',
 'estadia maravilhoso belissimo ver praia falar cida otimo cozinheira notar risoto ',
 'adorar cida querido EMOJI ',
 'apartamento localizar possuir vista mar privilegiada atendimento Flavio cida impecavel super indico ',
 'Flavio cida gentis apartamento realmente ficar frente praia lir jacuzzi nao aquecer porem proprietario dizer regularizar ',
 'apartamento aconchegante otimo localizacao funcionaria cido maravilhoso atencioso cozinha extremamente ficar satisfeito ',
 'flavio atencioso cidar ajuda periodo estadia formidavel ',
 'familia amamos apartamento sr flavio localizacao maravilhoso vista lindo jacuzzi churrasqueira mesa bilhar nao precisar sair apartamento divertir cida empregado sr flavio educado pontual cozinha divertido voltar futuro ',
 'perfeito cobertura localizacao espetacular conforme planejar ver espetacular bom pessoalmente opiniao melhor foto airbnb estao parabens cuidado receptividade cida recomendar assunto amigo gu

####  Teste Stanza

In [ ]:
%%time
########## Stanza defined's processors for this task:
# tokenize - Segments a Document into Sentences, each containing a list of Tokens.
# mwt - Expands multi-word tokens (MWTs) into multiple words when they are predicted by the tokenizer.Fix contractions.
# pos - UPOS, XPOS, and UFeats annotations are accessible through Word’s properties `pos`, `xpos`, and `ufeats`.
# lemma - Perform lemmatization on a Word using the `Word.text` and `Word.upos` values. The result can be accessed as `Word.lemma`. 
nlp = stanza.Pipeline(lang='pt', processors='tokenize,mwt,pos,lemma', use_gpu=True, download_method=None)
postprocess = []
for rawtxt in arr_texts[204]:
  if len(rawtxt):
    postprocess.append(preprocess_txt(rawtxt, nlp_obj=nlp, lemma=True))

postprocess

INFO:stanza:Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


CPU times: user 3.59 s, sys: 1.16 s, total: 4.76 s
Wall time: 10.6 s


['cobertura panoramico guaruja pitangueiro com spa ',
 'nosso estadia ser maravilhoso belissimo ver de o praia sem falar em o cida otimo cozinheira nota para risoto ',
 'adorar tudo e o cida um querido EMOJI ',
 'o apartamento este bem localizar possuir um vista para mar privilegiar o atendimento de o flavio e de o cida ser impecavel super indicar ',
 'flavio e cida ser muito gentil o apartamento e realmente bom e ficar em frente o praia e lir a jacuzzi em ao estar aquecer porem o proprietario dizer estar regularizar ',
 'apartamento muito bom aconchegante otimo localizacao a funcionar cida e maravilhoso atencioso e cozinha extremamente bem ficar muito satisfeito ',
 'flavio muito atencioso e cido quem nós ajudar em o periodo de o estadia e formidavel ',
 'eu e meu familia amar o apartamento de o sr flavio localizacao maravilhoso ver lindo e com o jacuzzi churrasqueiro e mesa de bilhar em ao precisar sair de o apartamento para nós divertir o cida empregar de o sr flavio e muito educado

### Step 06 - Applying preprocessing operations on dataframe text columns

In [ ]:
%%time
df_txt_cols_processed = df_txt_cols.applymap(preprocess_txt, na_action='ignore', nlp_obj=nlp, lemma=True)

CPU times: user 5min 11s, sys: 649 ms, total: 5min 12s
Wall time: 5min 14s


In [ ]:
df_txt_cols_processed.sample(5)

,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments,reviews/10/comments,reviews/11/comments,reviews/12/comments,reviews/13/comments,reviews/14/comments,reviews/15/comments,reviews/16/comments,reviews/17/comments,reviews/18/comments,reviews/19/comments
68,apartamento dorm guarujo lado praia piscina,NaN,apartamento show de bola tudo com facil disposicao perto de o todo o comercio desde farmacia ate fastfood anfitriao super atencioso educar e gente finissimo ir retornar e indicar familiar e amigo muito obrigado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,guarujo apartamento ver para o mar,NaN,apartamento excelente com um lindo vista de o mar localizar em um praia muito bom o condominio ter piscina aquecer sauno e muito espaco,tudo perfeito apartamento limpo espacoso aconchegante condominio maravilhoso ser tudo otimo pretender voltar novamente,o apartamento e muito bonito limpo e renovar ter ar condicionar em todo o comando ficar chateado que o piscina que mostrar em o foto que ficar em o sacada este de reforma e o churrasqueira tb nao poder utilizar entretanto nao atrapalhar o estadia o anfitria sempre responder em o hora o condomino ter um estrutura otimo com seguranca e lazer o praia e lindo com pouco ondas perfeito pra crianca,localizacao otimo vista lindo bom que o foto de o anuncio super limpo e organizar local agradavel otimo acomodar amar super recomendar,um de o bom estadia em que ja ficar o apartamento e excelente e o condominio incrivel recomendar de olho fechar,o apto e otimo o vista maravilhoso o condominio e muito gostoso super indicar,o apartamento e excelente muito bom que em o foto em o verdade o ivanilde e um otimo anfitrir e tudo correr muito bem com certeza voltar,tudo clean e organizar,o apartamento e incrivel o vista e o localidade e maravilhoso assim como o lindo praia privativo que e estonteante o ivanilde ser prestativo e simpatico para que o nosso final de semana se tornar inesquecivel pretender voltar o mais breve possivel obrigar por tudo,o apartamento e o condominio ser incrivel eu e meu familia amar o funcionarios de o condominio ser super atencioso o que tornar tudo muito melhor o vista de o apartamente e indescritivel o apartamento e lindissimo nao vir o hora de voltar,o vista realmente e incrivel local privilegiar o servico de jipe para o praia e muito legal e pratico ainda contar com servico exclusivo de guardasol e bar em o praia adorar o nosso estadia super valer o pena,ser um fin de semana increivel o apto e super bem localizar com um vista increivel para o mar o apto e super limpo moderno e bonito tambem ter acesso a um praia privado increivel ter piscina ter tudo para so ficar ahi tudo o fin de semanaobrigar ivanilde,otimo apartamento acomoder pessoa tranquilamente o anfitria sempre prestativo e atender nosso solicitaco rapidamente,o apartamento e incrivel o vista e um de o mais belo de o guaruja sem contar o atendimento de o anfitrio certamente ir retornar,apartamento otimo com um vista para o mar incrivel o condominio e muito bom tambem adorar o servico de jeep para o praia,otimo anfitria otimo apto indicar muito nao ha detalhe,o apartamento e perfeito o acomodaco em o geral ser otimo tudo muito limpo e arrumar decoracao armonioso sem contar o vista que e maravilhoso e dispensar comentario condominar tambem muito organizar com funcionarios muito educado tudo muito bom esperar poder voltar um dia obrigar mais uma vez ivanilde por o atencao,o apartamento e bem espaco acomoder pessoa com conforto o vista de o varanda e arrasador e o piscina tambem em o varanda e otimo o condominio oferecer bastante opco de entretenimento como sala de jogo sala fitne gaiola com passaro o crianca adorar piscina externo e piscina aquecer e transporte para o praia de o condominio se querer ir por o trilha tambem e legal

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

tqdm.pandas()

def parallel_applymap(df, func, worker_count, **kwargs):
    def _apply(shard):
        return shard.progress_applymap(func, **kwargs)

    shards = np.array_split(df, worker_count)
    with ThreadPoolExecutor(max_workers=worker_count) as e:
        futures = e.map(_apply, shards)
    return pd.concat(list(futures))

In [ ]:
%%time
df_txt_cols_processed = parallel_applymap(df_txt_cols, preprocess_txt, worker_count=10, na_action='ignore', nlp_obj=nlp, lemma=True)

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1188 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

...

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1188 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1188 [00:00<?, ?it/s]

CPU times: user 8min 24s, sys: 1min 14s, total: 9min 39s
Wall time: 6min 57s


In [ ]:
df_txt_cols_processed

,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments,reviews/10/comments,reviews/11/comments,reviews/12/comments,reviews/13/comments,reviews/14/comments,reviews/15/comments,reviews/16/comments,reviews/17/comments,reviews/18/comments,reviews/19/comments
0,ap q conforto e comodidade,NaN,the host canceled this reservation de a<unk>s before arrival this is an automated posting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,apartamento guarujo praia enseada sp grupo rei,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,casa em o praia de pernambuco guarujo sp,NaN,o casa ficar bem proximo de o praia alem de isso o casa e um graca tudo muito limpo e organizar o anfritrio ser muito atencioso recomendar voltar mais vez,um lugar lir e proximar o praia de ela para ir de o pe cristian e dona mario super atencioso e cuidadoso conosco casa super limpa e com utensilio que de ela para cozinhar sem problema o praia e de agua limpa e bem familiar amar o lugar e pretender voltar mais vez valer muito o pena,cristian e dona mario excelente anfitrio super flexibilizar o check out e gostar de ter ficar mais rs,casa extremamente limpa e muito bem cuidado igualzinho o foto sem falar em o hospitalidade de o dona mario ela e amor com certeza voltar,lugar muito limpo e casa bastante espacoso o valor tambem e otimo por o que e oferecer o cristian ser muito prestativo e nós ajudar com o check in ajustar para o horario que pretender chegar e sair recomendar,otimo local muito bem localizar super proximo de o praia de o eden iporanga e ainda mais de o praia de pernambucofoi super flexivel com o horario de entrada e saidamuito limpo organizar e bonito,super recomendar casa muito bom muito limpa dono mario um amor de pessoa esperar volta logo,o local corresponder o imagem apresentar e ser muito bem recepcionar o casa muito limpa e completo parabem a o anfitriao,otimo localizacao o cristian e o d mario ser otimo anfitrio,lugar simples e tranquilo cristian fazer um bom trabalho manter o contato comigo,lugar limpo arrumar e arejar recepcao formidavel bom localizacao com todo o item de o anuncio recomendar e voltar quando possivel,espaco limpo arejar um excelente recepcao e dia mario cuidar de nós como um mae pretender voltar em breve,excelente,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,vista maravilhoso,ser servidor publico e ter o grande prazer em recebelo em nosso cidade fundar em de janeiro de ela deter o titulo de celula matter de o nacionalidade brasileiro considerar primeiro vila de o brasil briga por o titulo de primeiro cidade ser bem vir e ter um otimo estadia,excelente estadia tudo limpo e bem arejar nao ter estacionamento existir um vaga de o lado de fora de o predio mas nunca ficar vaga mas isso e o de menos pois o localizacao e maravilhoso perto de tudo comercio praia banco tudo mesmo com certeza voltar,nós atender super bem ser bem atencioso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chale de o caruso,NaN,otimo estadia,o acomodacao e bom mas o acesso e complicado por um rua de terra que estar esburacado devido a chuva,muito bom,passar o feriado de o dia de o crianca ela e ser perfeito tudo organizar limpo e agradavel certeza que voltar,hospedar com o janine e algo que todo viagem precisar lugar tranquilo estrutura impecavel o chale contar com tudo de o bom e de o bom desde um bom wifi um tv smart grande utensilio domestico de primeiro qualidade cama magnifico com bom jogo de cama bem como um otimo churrasqueira ser com familia crianca e tudo mais todo adorar aproveitar muito nosso estadia fora que o condominio catarino home club e fantastico e ja e segundo vez que eu hosper em ele so que em o chale de o janine ser muito bom que em o primeiro haver ver como ja mencionar seu estrutura ser formidavel voce em ao 